# Citation

Much of the code and examples are copied/modified from 

> Blueprints for Text Analytics Using Python by Jens Albrecht, Sidharth Ramachandran, and Christian Winkler (O'Reilly, 2021), 978-1-492-07408-3.
>

- https://github.com/blueprints-for-text-analytics-python/blueprints-text
- https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch01/First_Insights.ipynb

---

# Setup

In [2]:
%run "/code/source/config/notebook_settings.py"

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
from source.library.text_analysis import count_tokens, tf_idf, get_context_from_keyword, \
    count_keywords, count_keywords_by, impurity

In [5]:
with Timer("Loading Data"):
    path = 'artifacts/data/processed/reddit.pkl'
    df = pd.read_pickle(path)

2023-02-25 18:23:20 - INFO     | Timer Started: Loading Data
2023-02-25 18:23:20 - INFO     | Timer Finished: (0.18 seconds)


---

# Exploratory Data Analysis

This section provides a basic exploration of the text and dataset.

## Dataset Summary

In [6]:
df.head(1)

,id,subreddit,title,post,impurity,post_clean,all_lemmas,partial_lemmas,bi_grams,adjs_verbs,nouns,noun_phrases,entities,post_length,num_tokens,language
0,74qv99,Honda,J32A3 Block with J35Z2 Crank and Rods?,"Hello, <lb><lb>I have my J32A3 egine open, ready to put new rings and all. I would like to know if I can swap the J35Z2 crank and rods from an Accord Sedan v6 3.5. I would like to keep my pistons, just change the rods and crank to have a little more displacement. This is a doable option? there is something I need to know first? This is for my Acura TL 3G 2004 Manual Transmission. I really would like to know if it's possible and if possible what differences are between then J35Z2 and let's say the classic J35A3.<lb><lb>Thanks in advance.",0.01,"Hello, I have my J32A3 egine open, ready to put new rings and all. I would like to know if I can swap the J35Z2 crank and rods from an Accord Sedan v6 _NUMBER_ . I would like to keep my pistons, just change the rods and crank to have a little more displacement. This is a doable option? there is something I need to know first? This is for my Acura TL 3G _NUMBER_ Manual Transmission. I really would like to know if it's possible and if possible what differences are between then J35Z2 and let's say the classic J35A3. Thanks in advance.","[hello, i, have, my, j32a3, egine, open, ready, to, put, new, ring, and, all, i, would, like, to, know, if, i, can, swap, the, j35z2, crank, and, rod, from, an, accord, sedan, v6, _number_, i, would, like, to, keep, my, piston, just, change, the, rod, and, crank, to, have, a, little, more, displacement, this, be, a, doable, option, there, be, something, i, need, to, know, first, this, be, for, my, acura, tl, 3, g, _number_, manual, transmission, i, really, would, like, to, know, if, it, be, possible, and, if, possible, what, difference, be, between, then, j35z2, and, let, us, say, ...]","[hello, j32a3, egine, open, ready, new, ring, like, know, swap, j35z2, crank, rod, accord, sedan, v6, _number_, like, piston, change, rod, crank, little, displacement, doable, option, need, know, acura, tl, 3, g, _number_, manual, transmission, like, know, possible, possible, difference, j35z2, let, classic, j35a3, thank, advance]","[j32a3 egine, egine open, new ring, j35z2 crank, accord sedan, sedan v6, v6 _number_, doable option, acura tl, tl 3, 3 g, g _number_, _number_ manual, manual transmission, classic j35a3]","[open, ready, new, like, know, swap, like, change, little, doable, need, know, like, know, possible, possible, let, classic]","[j32a3, egine, ring, j35z2, crank, rod, accord, sedan, v6, _number_, piston, rod, crank, displacement, option, acura, tl, g, _number_, manual, transmission, difference, j35z2, j35a3, thank, advance]","[egine open, new ring, j35z2 crank, doable option, classic j35a3]","[accord sedan (PRODUCT), first (ORDINAL), acura (ORG), 3 (CARDINAL), g _number_ manual transmission (ORG)]",542,46,English


### Numeric Summary

In [8]:
hlp.pandas.numeric_summary(df)

,# of Non-Nulls,# of Nulls,% Nulls,# of Zeros,% Zeros,Mean,St Dev.,Coef of Var,Skewness,Kurtosis,Min,10%,25%,50%,75%,90%,Max
impurity,"5,000",0,0.0%,"1,023",20.0%,0.0,0.0,1.0,1.9,7.8,0.0,0.0,0.0,0.0,0.0,0.0,0.2
post_length,"5,000",0,0.0%,0,0.0%,679.1,452.5,0.7,2.5,9.2,256.0,307.0,381.0,538.0,812.2,"1,217.1","4,174.0"
num_tokens,"5,000",0,0.0%,0,0.0%,54.2,36.3,0.7,2.6,9.9,15.0,25.0,31.0,43.0,65.0,96.0,378.0


### Non-Numeric

In [ ]:
hlp.pandas.non_numeric_summary(df)

---

### Examples

In [ ]:
df['post'].iloc[0][0:1000]

In [ ]:
'|'.join(df['partial_lemmas'].iloc[0])[0:1000]

In [ ]:
'|'.join(df['bi_grams'].iloc[0])[0:1000]

In [ ]:
'|'.join(df['noun_phrases'].iloc[0])[0:1000]

## Explore Non-Text Columns

#### Impurity

In [ ]:
ax = df['impurity'].plot(kind='box', vert=False, figsize=(10, 1))
ax.set_title("Distribution of Post Impurity")
ax.set_xlabel("Impurity")
ax.set_yticklabels([])
ax;

In [ ]:
df[['impurity', 'post', 'post_clean']].sort_values('impurity', ascending=False).head()

In [ ]:
df['language'].value_counts(ascending=False)

In [ ]:
df['subreddit'].value_counts(ascending=False)

Explore idiosyncrasies of various columns, e.g. same speaker represented multiple ways.

---

## Explore Text Column|

### Top Words Used

In [ ]:
remove_tokens = {'_number_', 'car'}
count_tokens(df['partial_lemmas'], remove_tokens=remove_tokens).head(10)

---

### Distribution of Text Length

In [ ]:
ax = df['post_length'].plot(kind='box', vert=False, figsize=(10, 1))
ax.set_title("Distribution of Post Length")
ax.set_xlabel("# of Characters")
ax.set_yticklabels([])
ax;

In [ ]:
ax = df['post_length'].plot(kind='hist', bins=60, figsize=(10, 2));
ax.set_title("Distribution of Post Length")
ax.set_xlabel("# of Characters")
ax;

In [ ]:
import seaborn as sns
sns.displot(df['post_length'], bins=60, kde=True, height=3, aspect=3);

In [ ]:
where = df['subreddit'].isin([
    'Lexus', 
    'mercedes_benz',
    'Audi',
    'Volvo',
    'BMW',
])
g = sns.catplot(data=df[where], x="subreddit", y="post_length", kind='box')
g.fig.set_size_inches(6, 3)
g.fig.set_dpi(100)
g = sns.catplot(data=df[where], x="subreddit", y="post_length", kind='violin')
g.fig.set_size_inches(6, 3)
g.fig.set_dpi(100)

### Word Frequency

In [ ]:
counts_df = count_tokens(df['partial_lemmas'], remove_tokens=remove_tokens)

In [ ]:
def plot_wordcloud(frequency_dict):
    wc = wordcloud.WordCloud(background_color='white',
        #colormap='RdYlGn',
        colormap='tab20b',
        width=round(hlpp.STANDARD_WIDTH*100),
        height=round(hlpp.STANDARD_HEIGHT*100),
        max_words = 200, max_font_size=150,
        random_state=42
    )
    wc.generate_from_frequencies(frequency_dict)

    fig, ax = plt.subplots(figsize=(hlpp.STANDARD_WIDTH, hlpp.STANDARD_HEIGHT))
    ax.imshow(wc, interpolation='bilinear')
    #plt.title("XXX")
    plt.axis('off')

In [ ]:
plot_wordcloud(counts_df.to_dict()['frequency']);

### TF-IDF

In [ ]:
tf_idf_lemmas = tf_idf(
    df=df,
    tokens_column='partial_lemmas',
    segment_columns = None,
    min_frequency_corpus=20,
    min_frequency_document=20,
    remove_tokens=remove_tokens,
)
tf_idf_lemmas.head()

In [ ]:
remove_tokens_bi_grams = {'_number_ year', '_number_ _number_', 'hey guy'}
tf_idf_bi_grams = tf_idf(
    df=df,
    tokens_column='bi_grams',
    segment_columns = None,
    min_frequency_corpus=20,
    min_frequency_document=20,
    remove_tokens=remove_tokens_bi_grams,
)
tf_idf_bi_grams.head()

In [ ]:
tf_idf_nouns = tf_idf(
    df=df,
    tokens_column='nouns',
    segment_columns = None,
    min_frequency_corpus=20,
    min_frequency_document=20,
    remove_tokens=remove_tokens,
)
tf_idf_nouns.head()

In [ ]:
tf_idf_noun_phrases = tf_idf(
    df=df,
    tokens_column='noun_phrases',
    segment_columns = None,
    min_frequency_corpus=20,
    min_frequency_document=20,
    remove_tokens=remove_tokens_bi_grams,
)
tf_idf_noun_phrases.head()

---

In [ ]:
ax = tf_idf_lemmas.head(30)[['tf-idf']].plot(kind='barh', width=0.99)
ax.set_title("TF-IDF of Uni-Grams")
ax.set_xlabel("TF-IDF")
ax.invert_yaxis();

In [ ]:
ax = tf_idf_bi_grams.head(30)[['tf-idf']].plot(kind='barh', width=0.99)
ax.set_title("TF-IDF of Bi-Grams")
ax.set_xlabel("TF-IDF")
ax.invert_yaxis();

In [ ]:
ax = tf_idf_nouns.head(30)[['tf-idf']].plot(kind='barh', width=0.99)
ax.set_title("TF-IDF of Bi-Grams")
ax.set_xlabel("TF-IDF")
ax.invert_yaxis();

In [ ]:
ax = tf_idf_noun_phrases.head(30)[['tf-idf']].plot(kind='barh', width=0.99)
ax.set_title("TF-IDF of Bi-Grams")
ax.set_xlabel("TF-IDF")
ax.invert_yaxis();

In [ ]:
plot_wordcloud(tf_idf_lemmas.to_dict()['tf-idf']);

In [ ]:
plot_wordcloud(tf_idf_bi_grams.to_dict()['tf-idf']);

#### By Subreddit

In [ ]:
remove_tokens_subreddit = set(df.subreddit.str.lower().unique())
remove_tokens_subreddit

In [ ]:
tf_idf_lemmas_per_sub = tf_idf(
    df=df,
    tokens_column='partial_lemmas',
    segment_columns = 'subreddit',
    min_frequency_corpus=10,
    min_frequency_document=10,
    remove_tokens=remove_tokens | remove_tokens_subreddit 
)
tf_idf_lemmas_per_sub.head(5)

In [ ]:
tf_idf_bigrams_per_sub = tf_idf(
    df=df,
    tokens_column='bi_grams',
    segment_columns = 'subreddit',
    min_frequency_corpus=10,
    min_frequency_document=10,
    remove_tokens=remove_tokens_bi_grams
)
tf_idf_bigrams_per_sub.head(5)

In [ ]:
tf_idf_nouns_per_sub = tf_idf(
    df=df,
    tokens_column='nouns',
    segment_columns = 'subreddit',
    min_frequency_corpus=10,
    min_frequency_document=10,
    remove_tokens=remove_tokens | remove_tokens_subreddit
)
tf_idf_nouns_per_sub.head(5)

In [ ]:
tf_idf_nounphrases_per_sub = tf_idf(
    df=df,
    tokens_column='noun_phrases',
    segment_columns = 'subreddit',
    min_frequency_corpus=10,
    min_frequency_document=10,
    remove_tokens=remove_tokens_bi_grams
)
tf_idf_nounphrases_per_sub.head(5)

---

In [ ]:
tokens_to_show = tf_idf_lemmas_per_sub.query("subreddit in ['Lexus', 'Volvo']").reset_index()
tokens_to_show.head()

In [ ]:
px.bar(
    tokens_to_show.groupby(['subreddit']).head(20).sort_values('tf-idf', ascending=True),
    x='tf-idf',
    y='token',
    color='subreddit',
    barmode='group',
    title="Top 20 Lemmas for Volvo & Lexus"
)

In [ ]:
tokens_to_show = tf_idf_bigrams_per_sub.query("subreddit in ['Lexus', 'Volvo']").reset_index()
tokens_to_show.head()

In [ ]:
px.bar(
    tokens_to_show.groupby(['subreddit']).head(20).sort_values('tf-idf', ascending=True),
    x='tf-idf',
    y='token',
    color='subreddit',
    barmode='group',
    title="Top 20 Bi-Grams for Volvo & Lexus"
)

---

In [ ]:
get_context_from_keyword(df.query("subreddit == 'Lexus'")['post'], keyword='think')

In [ ]:
get_context_from_keyword(df.query("subreddit == 'Volvo'")['post'], keyword='think')

##### Lexus

In [ ]:
tokens_to_show = tf_idf_lemmas_per_sub.query("subreddit == 'Lexus'").reset_index()
#tokens_to_show = tokens_to_show[~tokens_to_show.token.isin(stop_words)]
tokens_to_show = tokens_to_show[['token', 'tf-idf']].set_index('token')
tokens_to_show = tokens_to_show.to_dict()['tf-idf']
plot_wordcloud(tokens_to_show);

##### Volvo

In [ ]:
tokens_to_show = tf_idf_lemmas_per_sub.query("subreddit == 'Volvo'").reset_index()
#tokens_to_show = tokens_to_show[~tokens_to_show.token.isin(stop_words)]
tokens_to_show = tokens_to_show[['token', 'tf-idf']].set_index('token')
tokens_to_show = tokens_to_show.to_dict()['tf-idf']
plot_wordcloud(tokens_to_show);

### Keywords in Context

In [ ]:
contexts = get_context_from_keyword(
    documents=df['post'],
    window_width=50,
    keyword='replac',
    num_samples = 20,
    random_seed=42
)
for x in contexts:
    print(x)

---